In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import typing
from pathlib import Path
from tempfile import TemporaryDirectory

import matplotlib as mpl
import pandas as pd
import seaborn as sns
import torch
import wandb
from matplotlib import pyplot as plt
from scipy.ndimage import gaussian_filter

import opf
import opf.dataset
import opf.utils
from opf.hetero import HeteroSage
from opf.modules import OPFDual
from opf.dataset import PowerflowData

mpl.rcParams["text.usetex"] = False

sns.set_style("white")
sns.set_context("paper")

torch.set_float32_matmul_precision("high")

In [ ]:
run_id = "c7wu8c0z"
run_uri = f"wandb://damowerko/opf/{run_id}"

In [ ]:
base, path = run_uri.split("://")
if base != "wandb":
    raise ValueError("Only wandb runs are supported")

api = wandb.Api()
run = api.run(path)
run.config.update(data_dir="../data")

# initialize data
config = run.config
# config.update(batch_size=128)

# override some config values
config["case_name"] = "case118_ieee"
dm = opf.dataset.CaseDataModule(**config)
dm.setup()
assert dm.test_dataset is not None

gcn = HeteroSage(
    dm.metadata(),
    in_channels=max(dm.feature_dims.values()),
    out_channels=4,
    **run.config,
)
model = OPFDual(gcn, **run.config).cuda()

# load checkpoint
with TemporaryDirectory() as tmpdir:
    artifact = api.artifact(f"damowerko/opf/model-{run.id}:best")
    checkpoint_path = artifact.download(root=tmpdir)
    checkpoint = torch.load(Path(checkpoint_path) / "model.ckpt", map_location="cpu")
    state_dict = checkpoint["state_dict"]
    state_dict = {k.replace("._orig_mod", ""): v for k, v in state_dict.items()}
    model.load_state_dict(state_dict, strict=True)

In [ ]:
save_dir = Path(f"../figures/dual/{run.config['case_name']}/")
save_dir.mkdir(parents=True, exist_ok=True)


def save(title: str, fig=plt):
    # fig.savefig(save_dir / f"{title}.png")
    fig.savefig(save_dir / f"{title}.pdf")

In [ ]:
def data_to_device(data: PowerflowData, device: str):
    return PowerflowData(*(x.to(device=device) for x in data))  # type: ignore

In [ ]:
import h5py
from pathlib import Path
from tqdm.notebook import tqdm

data_path = Path(f"../data/out/test/{run_id}_test.parquet")
data_path.parent.mkdir(parents=True, exist_ok=True)

if not data_path.exists():
    metrics = []
    with torch.no_grad():
        for batch in tqdm(dm.test_dataloader()):
            batch = data_to_device(batch, "cuda")
            variables = model(batch)[0]
            # variables = model.project_powermodels(
            #     variables, batch.powerflow_parameters, clamp=True
            # )
            for i in range(batch.data.num_graphs):
                _variables = variables[i]
                cost = model.cost(_variables, batch.powerflow_parameters)
                constraints = model.constraints(
                    _variables, batch.powerflow_parameters, None
                )
                metrics.append(
                    {
                        k: v.item()
                        for k, v in model.metrics(
                            cost, constraints, "test", True
                        ).items()
                    }
                )
    df = pd.DataFrame(metrics)
    df.to_parquet(data_path)
else:
    df = pd.read_parquet(data_path)

In [ ]:
with h5py.File(Path(batch.powerflow_parameters.casefile).with_suffix(".h5"), "r") as f:
    df["acopf/cost"] = f["objective"][-1000:] / batch.powerflow_parameters.reference_cost  # type: ignore

In [ ]:
df["test/error_max"] = df[["test/equality/error_max", "test/inequality/error_max"]].max(
    axis=1
)

# Summary

In [ ]:
display(
    df[
        [
            "test/error_max",
            "test/equality/error_max",
            "test/equality/bus_active_power/error_max",
            "test/equality/bus_reactive_power/error_max",
            "test/inequality/error_max",
            "test/inequality/active_power/error_max",
            "test/inequality/reactive_power/error_max",
            "test/inequality/voltage_magnitude/error_max",
            "test/inequality/forward_rate/error_max",
            "test/inequality/backward_rate/error_max",
            # "test/inequality/voltage_angle_difference/error_max",
        ]
    ].max()
)

print(
    f"""
GNN Cost: {df["test/cost"].mean():0.4f}
IPOPT Cost: {df["acopf/cost"].mean():0.4f}
GNN/IPOPT: {(df["test/cost"]/df["acopf/cost"]).mean():0.4f}

Maximum equality violation rate: {df["test/equality/rate"].max():0.4f}
Maximum inequality violation rate: {df["test/inequality/rate"].max():0.4f}
Rate of any violation: {((df["test/error_max"] > 0.01)).sum() / len(df):0.4f}
"""
)

# Histograms

In [ ]:
from opf.utils import FlowLayout

aspect = 1.618
ylabel = "Count / # of samples"
kwargs = dict(
    bins=10,
    stat="proportion",
    aspect=aspect,
    height=3.5 / aspect,
)


sns.displot(df, x="test/inequality/error_max", **kwargs)
plt.xlabel("Max inequality error")
plt.ylabel(ylabel)
save("error_max")

sns.displot(df, x="test/inequality/error_mean", **kwargs)
plt.xlabel("Mean inequality error")
plt.ylabel(ylabel)
save("error_mean")

# Cost improvement
df["test/cost/improvement"] = df["test/cost"] / df["acopf/cost"]
df["violation"] = df["test/error_max"] > 0.01
sns.displot(df[~df["violation"]], x="test/cost/improvement", **kwargs)
plt.xlabel("GNN / IPOPT cost ratio")
plt.ylabel(ylabel)
save("costs")

# map variable names to series names
fmt = "test/inequality/%s/error_max"
hist_dict = {
    "equality": ["test/equality/bus_active_power/error_max"],
    "gen": [fmt % "active_power", fmt % "reactive_power"],
    "vm": [fmt % "voltage_magnitude"],
    "rate": [fmt % "forward_rate", fmt % "backward_rate"],
    # "vad": [fmt % "voltage_angle_difference"],
}

sns.displot(df["test/equality/bus_active_power/error_max"], **kwargs)
plt.xlabel("Bus power equality error")
plt.ylabel(ylabel)
save("error_equality")

power_df = df.melt(value_vars=[fmt % "active_power", fmt % "reactive_power"])
sns.displot(power_df, x="value", **kwargs)
plt.xlabel("Generated power error")
plt.ylabel(ylabel)
save("error_gen")

sns.displot(df[fmt % "voltage_magnitude"], **kwargs)
plt.xlabel("Voltage magnitude error")
plt.ylabel(ylabel)
save("error_vm")

flow_df = df.melt(value_vars=[fmt % "forward_rate", fmt % "backward_rate"])
sns.displot(flow_df, x="value", **kwargs)
plt.xlabel("Power rate limit error")
plt.ylabel(ylabel)
save("error_rate")

# sns.displot(df[fmt % "voltage_angle_difference"], **kwargs)
# plt.xlabel("Voltage angle difference error")
# plt.ylabel(ylabel)
# save("error_vad")

FlowLayout().all_open()

# Visualizing Violations

In [ ]:
sort_term = "test/error_max"
quantile = 0.5

s = df[sort_term]
index = int((s.sort_values()[::-1] <= s.quantile(quantile)).idxmax())
print(sort_term, s[index])
print("Worst Violation Index", index)
df.iloc[index][
    [
        "test/cost",
        "acopf/cost",
        "test/equality/bus_active_power/error_max",
        "test/equality/bus_reactive_power/error_max",
        "test/equality/bus_reference/error_max",
        "test/inequality/error_max",
        "test/inequality/active_power/error_max",
        "test/inequality/reactive_power/error_max",
        "test/inequality/voltage_magnitude/error_max",
        "test/inequality/forward_rate/error_max",
        "test/inequality/backward_rate/error_max",
        # "test/inequality/voltage_angle_difference/error_max",
    ]
]

In [ ]:
# perform inference on the test set
from opf.plot import plot_constraints
import opf.powerflow as pf


with torch.no_grad():
    dm.setup("test")
    dataset = dm.test_dataset
    data = dataset[index]  # type: ignore
    data = data_to_device(data, "cuda")

    variables, Sf, St = model(data)
    variables, _, _ = model._step_helper(
        variables, data.powerflow_parameters, project_powermodels=True
    )
    # move variables and data back to cpu
    data = data_to_device(data, "cpu")
    variables = variables.cpu()

    constraints = pf.build_constraints(variables, data.powerflow_parameters)
    plots = plot_constraints(constraints)

for name in plots:
    name = typing.cast(str, name)
    save(f"constraint_{name.replace('/', '_')}", fig=plots[name])
FlowLayout().all_open()

# Training History

In [ ]:
key_map = {
    "/cost": "Cost",
    "/equality/error_mean": "Equality Error Mean",
    "/inequality/error_mean": "Inequality Error Mean",
    "/equality/error_max": "Equality Error Max",
    "/inequality/error_max": "Inequality Error Max",
}
train_df = run.history(
    samples=1000, keys=["train" + k for k in key_map], x_axis="epoch"
)
val_df = run.history(samples=1000, keys=["val" + k for k in key_map], x_axis="epoch")

for k in key_map:
    plt.figure()
    plt.xlabel("Epoch")
    plt.ylabel(key_map[k])

    train_y = train_df["train" + k]
    val_y = val_df["val" + k]

    train_y = gaussian_filter(train_y, sigma=5)
    val_y = gaussian_filter(val_y, sigma=5)

    plt.plot(train_df["epoch"], train_y, label="Train")
    plt.plot(val_df["epoch"], val_y, label="Validation")
    plt.ylim(0, 1.5)
    plt.legend()
    save(f"history{k.replace('/', '_')}")

FlowLayout().all_open()